In [1]:
import pandas as pd
import numpy as np
import pickle
import argparse
import sys
import os

### Following is code from RF_deploy.py

In [11]:
def gene_checker(input_file):
    '''
    Function checks the input file's genes.
    The intersection between the input file's genes and the predetermined classifier genes is taken.
    Then the function checks for genes are not present in the input file.
    For those genes, the expression vector is set to 0.
    A new dataframe is returned with the correct order of genes.

    Parameters
    ----------
    input_file; pandas df - shape should be (samples x genes)
    
    Returns
    ---------
    new_input_file; pandas df - shape (samples x genes)
    '''
    
    classifier_genes = np.loadtxt('../ClassifierGenes.txt', dtype='str')
    new_input_file = input_file.T.loc[classifier_genes].T # seleting classifier selected genes in the classifier determined order
    
    # will fill genes that do not exist in the input with zero
    # if no NAN values, none will be filled
    new_input_file = new_input_file.fillna(0) 
    
    return new_input_file

def deploy(input_file):
    classifier_genes = np.loadtxt('../ClassifierGenes.txt', dtype='str')

    expr_input = input_file
    
    print('reading input...') 
    expr_input = pd.read_csv(expr_input, sep='\t', index_col=0)

    print("before gene intersection...")
    print(expr_input.shape)
    expr_input = gene_checker(expr_input) # making sure genes are correct for classifier
    print("After gene intersection...")
    print(expr_input.shape)

    print('applying model...')
    model = pickle.load(open('../RiboVsPoly.sav', 'rb'))
    print(model)
    
    predictions = model.predict(expr_input)
    predict_proba = model.predict_proba(expr_input)
    
    print("saving results...")
    predictions = pd.DataFrame(np.hstack([predictions.reshape(-1,1), predict_proba]), index=expr_input.index, columns=['Ribo', 'Proba_0', 'Proba_1'])

    return predictions

## Applying on openPBTA polyA samples

In [12]:
predictions = deploy("../data/pbta-gene-expression-rsem-tpm.polya.csv")

reading input...
before gene intersection...
(58, 58347)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


After gene intersection...
(58, 25924)
applying model...


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.

RandomForestClassifier(bootstrap=True, ccp_alpha=None, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=1,
                       warm_start=False)


[Parallel(n_jobs=15)]: Done 420 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done 770 tasks      | elapsed:    0.2s
[Parallel(n_jobs=15)]: Done 1000 out of 1000 | elapsed:    0.3s finished
[Parallel(n_jobs=15)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=15)]: Done 170 tasks      | elapsed:    0.1s
[Parallel(n_jobs=15)]: Done 420 tasks      | elapsed:    0.2s


saving results...


[Parallel(n_jobs=15)]: Done 770 tasks      | elapsed:    0.3s
[Parallel(n_jobs=15)]: Done 1000 out of 1000 | elapsed:    0.4s finished


In [13]:
# All samples were predicted to be polyA
predictions

,Ribo,Proba_0,Proba_1
BS_0VXZCRJS,0.0,0.622000,0.378000
BS_0ZA67BBC,0.0,0.654994,0.345006
BS_1N7MQZGR,0.0,0.645000,0.355000
BS_21ET39G7,0.0,0.668994,0.331006
BS_2JP7RBMB,0.0,0.660000,0.340000
BS_3AC3SRWH,0.0,0.737054,0.262946
BS_49CJNZ06,0.0,0.658994,0.341006
BS_4PPHAQXF,0.0,0.664000,0.336000
BS_4PWDGEB0,0.0,0.665000,0.335000
BS_58YXHGAJ,0.0,0.653000,0.347000
